In [1]:
import sys, time, random
import os
os.environ['EPICS_CA_AUTO_ADDR_LIST'] = 'no'
os.environ['EPICS_CA_ADDR_LIST'] = '127.0.0.1'
sys.path.append('../..')
import pybeamtools

[INFO ][MainThread][06:05:48.497 logging.py  23] Starting shared logging thread on PID 18132
[INFO ][log_listen][06:05:48.498 logging.py  32] Logging listener started on PID 18132
[INFO ][MainThread][06:05:48.500 logging.py  27] Logging setup finished


In [2]:
# %load_ext autoreload
# %autoreload 2

In [3]:
def time_fun():
    return time.time()*0.5

t = 0.0
def fixed_time():
    return t

In [4]:
from pybeamtools.sim.core import SimulationEngine, SignalEngineOptions
sim = SimulationEngine(SignalEngineOptions(time_function=fixed_time))
sim.TRACE = True

[INFO ][MainThread][06:05:49.034   utils.py 148] Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[INFO ][MainThread][06:05:49.034   utils.py 160] NumExpr defaulting to 8 threads.
[DEBUG][MainThread][06:05:49.383 _backend.py  35] Backend 'array' registered
[DEBUG][MainThread][06:05:49.385 _backend.py  35] Backend 'numpy' registered
[DEBUG][MainThread][06:05:49.385 _backend.py  47] Selecting backend: 'numpy'
[DEBUG][MainThread][06:05:49.417    core.py 557] Starting update thread
[DEBUG][update_thr][06:05:49.418    core.py 531] Hello from update thread (id 11896)


In [5]:
from pybeamtools.sim.pddevices import *
ctx = SignalContext(se=sim)
echo = EchoDevice(ctx,options=EchoDeviceOptions(name='echo', data={'TEST:ECHO:1':5}))
echo2 = EchoDevice(ctx, options=EchoDeviceOptions(name='echo2', data={'TEST:ECHO:2':15}))
echo3= EchoDevice(ctx,options=EchoDeviceOptions(name='echo3', data={'TEST:ECHO:3':25}))
echo4 = EchoDevice(ctx, options=EchoDeviceOptions(name='echo4', data={'TEST:ECHO:4':35}))

In [6]:
var = 'TEST:GEN1'
data = {var:1.5}
def update_fun(t_sched, t_run, dep_data):
    return data

def read_fun(t_sched, t_run, channel_name):
    return data[var]

def write_fun(t_sched, t_run, value_dict):
    data[var] = value_dict[var]

def startup_fun(dev):
    import threading
    def updater():
        while True:
            data[var] = random.random()*10
            #print(f'Issuing update {data=}')
            for k in data:
                dev.ctx.issue_update(k)
            time.sleep(20)
    th = threading.Thread(target=updater, name='updater')
    th.daemon = True
    th.start()
gopt = GenericDeviceOptions(name='gen', update_fun=update_fun, read_fun=read_fun, write_fun=write_fun, startup_fun=startup_fun, channel_map={var:{}})

In [7]:
generic1 = GenericDevice(ctx, options=gopt)

[DEBUG][MainThread][06:05:49.533    core.py 236] Added device (gen) (device.channel_map={'TEST:GEN1': {}})
[DEBUG][update_thr][06:05:49.534    core.py 542] Update i=1 of ch TEST:GEN1 at 0.0
[DEBUG][update_thr][06:05:49.534    core.py 372] UP (gen): result ({'TEST:GEN1': 8.793390158614677})


In [8]:
sim.latest_data

{'TEST:GEN1': 8.793390158614677}

In [9]:
proxy1 = ProxyDevice(ctx, options=ProxyDeviceOptions(name='proxy1', channel_map={'TEST:PROXY:1':{var: TRIGSPEC.PROPAGATE}}))
proxy2 = ProxyDevice(ctx, options=ProxyDeviceOptions(name='proxy2', channel_map={'TEST:PROXY:2':{'TEST:ECHO:2': TRIGSPEC.IGNORE}}))
proxy3 = ProxyDevice(ctx, options=ProxyDeviceOptions(name='proxy3', channel_map={'TEST:PROXY:3':{'TEST:PROXY:1': TRIGSPEC.PROPAGATE}}))
proxy4 = ProxyDevice(ctx, options=ProxyDeviceOptions(name='proxy4', channel_map={'TEST:PROXY:4':{'TEST:PROXY:1': TRIGSPEC.PROPAGATE}}))

[DEBUG][MainThread][06:05:49.606    core.py 236] Added device (proxy1) (device.channel_map={'TEST:PROXY:1': {'TEST:GEN1': <TRIGSPEC.PROPAGATE: 1>}})
[DEBUG][MainThread][06:05:49.607    core.py 236] Added device (proxy2) (device.channel_map={'TEST:PROXY:2': {'TEST:ECHO:2': <TRIGSPEC.IGNORE: 2>}})
[DEBUG][MainThread][06:05:49.607    core.py 236] Added device (proxy3) (device.channel_map={'TEST:PROXY:3': {'TEST:PROXY:1': <TRIGSPEC.PROPAGATE: 1>}})
[DEBUG][MainThread][06:05:49.608    core.py 236] Added device (proxy4) (device.channel_map={'TEST:PROXY:4': {'TEST:PROXY:1': <TRIGSPEC.PROPAGATE: 1>}})


In [10]:
sim.latest_data

{'TEST:GEN1': 8.793390158614677,
 'TEST:PROXY:1': None,
 'TEST:PROXY:2': None,
 'TEST:PROXY:3': None,
 'TEST:PROXY:4': None}

In [11]:
from pybeamtools.controls import ConnectionOptions, Accelerator, AcceleratorOptions
from pybeamtools.controls import *
# ao = AcceleratorOptions(connection_settings=ConnectionOptions(network='dummy'))
# simacc = SimulationEngine(SignalEngineOptions(time_function=time.time))
# #simacc.TRACE = True
# simacc.start_scan_thread()
# acc = Accelerator(options=ao, ctx=simacc)
# pv = SimPV(options=PVOptions(name='X0', low=0.0, high=5.0, security=PVAccess.READWRITE))
# pv_exact = SimPV(options=PVOptions(name='OBJ0', security=PVAccess.READONLY))

ao = AcceleratorOptions(connection_settings=ConnectionOptions(network='epics'))
acc = Accelerator(options=ao)
# pv = EPICSPV(options=PVOptions(name='X0', low=0.0, high=5.0, security=PVAccess.READWRITE))
# pv_exact = EPICSPV(options=PVOptions(name='OBJ0', security=PVAccess.READONLY))
#acc.add_pv_object([pv, pv_exact])
print('---')

[INFO ][MainThread][06:05:49.682 control_lib.py  59] Control lib init
[INFO ][MainThread][06:05:49.683 network.py 132] Creating EPICS connection manager
[INFO ][MainThread][06:05:49.683 network.py 134] Context not provided, creating new caproto context
[DEBUG][   command][06:05:49.685  client.py 611] Broadcaster command loop is running.
[DEBUG][check_for_][06:05:49.687  client.py 729] Broadcaster check for unresponsive servers loop is running.
[DEBUG][MainThread][06:05:49.687 _broadcaster.py  94] RepeaterRegisterRequest(client_address='0.0.0.0')
[DEBUG][MainThread][06:05:49.690  _utils.py 530] Netifaces unavailable; using 127.0.0.1 as local address
[DEBUG][MainThread][06:05:49.690  client.py 410] 1 commands 16B
[DEBUG][  selector][06:05:49.691 _broadcaster.py 136] RepeaterConfirmResponse(repeater_address='127.0.0.1')
[DEBUG][     retry][06:05:49.693  client.py 842] Broadcaster search-retry thread has started.
[DEBUG][    search][06:05:49.694  client.py 1087] Context search-results proc

---


In [12]:
epics_x0ao = EPICSDevice(ctx, options=EPICSDeviceOptions(name='VARX0AO',pv_config=PVOptions(name='X0:AO', security=PVAccess.READONLY)), acc_context=acc)
epics_x0ai = EPICSDevice(ctx, options=EPICSDeviceOptions(name='VARX0AI',pv_config=PVOptions(name='X0:AI', security=PVAccess.READWRITE)), acc_context=acc)
epics_obj = EPICSDevice(ctx, options=EPICSDeviceOptions(name='OBJECTIVE',pv_config=PVOptions(name='OBJ0', security=PVAccess.READONLY)), acc_context=acc)

[DEBUG][MainThread][06:05:49.762 network.py 158] Adding 1 PV objects
[DEBUG][MainThread][06:05:49.763    core.py 236] Added device (VARX0AO) (device.channel_map={'VARX0AO': {}})
[DEBUG][     retry][06:05:49.763  client.py 882] Sending 1 SearchRequests
[DEBUG][MainThread][06:05:49.764 pddevices.py 204] EPICS device created: {'VARX0AO': {}}
[DEBUG][     retry][06:05:49.765 _broadcaster.py  94] VersionRequest(priority=0, version=13)
[DEBUG][MainThread][06:05:49.766 network.py 158] Adding 1 PV objects
[DEBUG][     retry][06:05:49.766 _broadcaster.py  92] SearchRequest(name='X0:AO', cid=29572, version=13, reply=5)
[DEBUG][MainThread][06:05:49.767    core.py 236] Added device (VARX0AI) (device.channel_map={'VARX0AI': {}})
[DEBUG][     retry][06:05:49.768  client.py 470] 2 commands 40B
[DEBUG][MainThread][06:05:49.768 pddevices.py 204] EPICS device created: {'VARX0AI': {}}
[DEBUG][     retry][06:05:49.769  client.py 882] Sending 1 SearchRequests
[DEBUG][MainThread][06:05:49.770 network.py 158

In [13]:
print(sim.channels_map_chain)
print(sim.channels_dep_chain)
print(sim.next_periodic_read_time)

[DEBUG][  selector][06:05:49.778 _broadcaster.py 136] VersionResponse(version=13)
[DEBUG][    search][06:05:49.778 _circuit.py 166] VersionRequest(priority=1, version=13)
[DEBUG][     retry][06:05:49.778 _broadcaster.py  92] SearchRequest(name='OBJ0', cid=29574, version=13, reply=5)
[DEBUG][  selector][06:05:49.780 _broadcaster.py 136] SearchResponse(port=5064, ip='255.255.255.255', cid=29573, version=13)
[DEBUG][    search][06:05:49.780 _circuit.py 166] HostNameRequest(name='SERVY')
[DEBUG][     retry][06:05:49.781  client.py 470] 2 commands 40B
[DEBUG][    search][06:05:49.783 _circuit.py 166] ClientNameRequest(name='boss')
[DEBUG][  selector][06:05:49.786 _broadcaster.py 136] VersionResponse(version=13)
[DEBUG][  selector][06:05:49.786 _broadcaster.py 136] SearchResponse(port=5064, ip='255.255.255.255', cid=29574, version=13)
[DEBUG][  selector][06:05:49.809  client.py 1484] VersionResponse(version=13)
[DEBUG][    search][06:05:49.811 _circuit.py 166] CreateChanRequest(name='X0:AO',

{'TEST:GEN1': {}, 'TEST:PROXY:1': {'TEST:GEN1': <TRIGSPEC.PROPAGATE: 1>}, 'TEST:PROXY:2': {'TEST:ECHO:2': <TRIGSPEC.IGNORE: 2>}, 'TEST:PROXY:3': {'TEST:PROXY:1': <TRIGSPEC.PROPAGATE: 1>}, 'TEST:PROXY:4': {'TEST:PROXY:1': <TRIGSPEC.PROPAGATE: 1>}, 'VARX0AO': {}, 'VARX0AI': {}, 'OBJECTIVE': {}}
{'TEST:GEN1': ['TEST:PROXY:1'], 'TEST:ECHO:2': [], 'TEST:PROXY:1': ['TEST:PROXY:3', 'TEST:PROXY:4']}
{}


In [14]:
sim.latest_data

[DEBUG][activate_s][06:05:49.920 _circuit.py 166] EventAddRequest(data_type=<ChannelType.TIME_DOUBLE: 20>, data_count=0, sid=1, subscriptionid=0, low=0.0, high=0.0, to=0.0, mask=13)
[DEBUG][activate_s][06:05:49.924 _circuit.py 166] EventAddRequest(data_type=<ChannelType.TIME_DOUBLE: 20>, data_count=0, sid=0, subscriptionid=1, low=0.0, high=0.0, to=0.0, mask=13)
[DEBUG][activate_s][06:05:49.924 _circuit.py 166] EventAddRequest(data_type=<ChannelType.TIME_DOUBLE: 20>, data_count=0, sid=2, subscriptionid=2, low=0.0, high=0.0, to=0.0, mask=13)
[DEBUG][  selector][06:05:49.929  client.py 2185] 'X0:AI': EventAddResponse(data=array([0.]), data_type=<ChannelType.TIME_DOUBLE: 20>, data_count=1, status=CAStatusCode(name='ECA_NORMAL', code=0, code_with_severity=1, severity=<CASeverity.SUCCESS: 1>, success=1, defunct=False, description='Normal successful completion'), subscriptionid=0, metadata=DBR_TIME_DOUBLE(status=<AlarmStatus.NO_ALARM: 0>, severity=<AlarmSeverity.NO_ALARM: 0>, timestamp=167508

{'TEST:GEN1': 8.793390158614677,
 'TEST:PROXY:1': None,
 'TEST:PROXY:2': None,
 'TEST:PROXY:3': None,
 'TEST:PROXY:4': None,
 'VARX0AO': 0.003604553541594932,
 'VARX0AI': 0.0,
 'OBJECTIVE': 1.0}

[DEBUG][  selector][06:05:52.476  client.py 2185] 'X0:AO': EventAddResponse(data=array([0.01406001]), data_type=<ChannelType.TIME_DOUBLE: 20>, data_count=1, status=CAStatusCode(name='ECA_NORMAL', code=0, code_with_severity=1, severity=<CASeverity.SUCCESS: 1>, success=1, defunct=False, description='Normal successful completion'), subscriptionid=1, metadata=DBR_TIME_DOUBLE(status=<AlarmStatus.NO_ALARM: 0>, severity=<AlarmSeverity.NO_ALARM: 0>, timestamp=1675080352.473025))
[DEBUG][update_thr][06:05:52.477    core.py 542] Update i=5 of ch VARX0AO at 0.0
[DEBUG][  selector][06:05:52.477  client.py 1484] EventAddResponse(data=array([0.01406001]), data_type=<ChannelType.TIME_DOUBLE: 20>, data_count=1, status=CAStatusCode(name='ECA_NORMAL', code=0, code_with_severity=1, severity=<CASeverity.SUCCESS: 1>, success=1, defunct=False, description='Normal successful completion'), subscriptionid=1, metadata=DBR_TIME_DOUBLE(status=<AlarmStatus.NO_ALARM: 0>, severity=<AlarmSeverity.NO_ALARM: 0>, timest

In [15]:
print(sim.channels_map_chain)
print(sim.channels_dep_chain)
print(sim.next_periodic_read_time)

{'TEST:GEN1': {}, 'TEST:PROXY:1': {'TEST:GEN1': <TRIGSPEC.PROPAGATE: 1>}, 'TEST:PROXY:2': {'TEST:ECHO:2': <TRIGSPEC.IGNORE: 2>}, 'TEST:PROXY:3': {'TEST:PROXY:1': <TRIGSPEC.PROPAGATE: 1>}, 'TEST:PROXY:4': {'TEST:PROXY:1': <TRIGSPEC.PROPAGATE: 1>}, 'VARX0AO': {}, 'VARX0AI': {}, 'OBJECTIVE': {}}
{'TEST:GEN1': ['TEST:PROXY:1'], 'TEST:ECHO:2': [], 'TEST:PROXY:1': ['TEST:PROXY:3', 'TEST:PROXY:4']}
{}


In [34]:
sim.write_channel('VARX0AI',random.random())

[DEBUG][MainThread][06:12:27.912    core.py 501] Writing (0.9702980380141308) to channel (VARX0AI)
[DEBUG][MainThread][06:12:27.913 control_lib.py 180] Individual PV validation passed
[INFO ][MainThread][06:12:27.913 control_lib.py 183] Write on (['X0:AI']) triggered (0) of (0) interlocks
[DEBUG][MainThread][06:12:27.913 control_lib.py 186] No interlocks triggered, skip polling stage
[DEBUG][MainThread][06:12:27.915 _circuit.py 166] WriteNotifyRequest(data=array([0.97029804]), data_type=<ChannelType.DOUBLE: 6>, data_count=1, sid=1, ioid=0, metadata=None)
[DEBUG][  selector][06:12:27.924  client.py 1484] ErrorResponse(original_request=MessageHeader(command=19, payload_size=8, data_type=6, data_count=1, parameter1=1, parameter2=0), cid=1, status=CAStatusCode(name='ECA_PUTFAIL', code=20, code_with_severity=160, severity=<CASeverity.WARNING: 0>, success=0, defunct=False, description='Channel write request failed'), error_message=bytearray(b'Python exception: AssertionError \x00\x00\x00\x00

CaprotoTimeoutError: Server at 127.0.0.1:5064 did not respond to attempt to write to channel named 'X0:AI' within 5.0-second timeout. The ioid of the expected response is 0.

[DEBUG][update_thr][06:12:33.384    core.py 372] UP (VARX0AO): result ({'VARX0AO': 0.9689299235970313})


[DEBUG][  selector][06:12:38.160  client.py 2185] 'X0:AO': EventAddResponse(data=array([0.99216992]), data_type=<ChannelType.TIME_DOUBLE: 20>, data_count=1, status=CAStatusCode(name='ECA_NORMAL', code=0, code_with_severity=1, severity=<CASeverity.SUCCESS: 1>, success=1, defunct=False, description='Normal successful completion'), subscriptionid=1, metadata=DBR_TIME_DOUBLE(status=<AlarmStatus.NO_ALARM: 0>, severity=<AlarmSeverity.NO_ALARM: 0>, timestamp=1675080758.158516))
[DEBUG][update_thr][06:12:38.161    core.py 542] Update i=107 of ch VARX0AO at 0.0
[DEBUG][  selector][06:12:38.161  client.py 1484] EventAddResponse(data=array([0.99216992]), data_type=<ChannelType.TIME_DOUBLE: 20>, data_count=1, status=CAStatusCode(name='ECA_NORMAL', code=0, code_with_severity=1, severity=<CASeverity.SUCCESS: 1>, success=1, defunct=False, description='Normal successful completion'), subscriptionid=1, metadata=DBR_TIME_DOUBLE(status=<AlarmStatus.NO_ALARM: 0>, severity=<AlarmSeverity.NO_ALARM: 0>, time

In [35]:
sim.latest_data

{'TEST:GEN1': 6.429969442923817,
 'TEST:PROXY:1': 6.429969442923817,
 'TEST:PROXY:2': None,
 'TEST:PROXY:3': 6.429969442923817,
 'TEST:PROXY:4': 6.429969442923817,
 'VARX0AO': 0.9768863963916476,
 'VARX0AI': 0.0,
 'OBJECTIVE': 1.0}

[DEBUG][  selector][06:12:48.182  client.py 2185] 'X0:AO': EventAddResponse(data=array([0.9646667]), data_type=<ChannelType.TIME_DOUBLE: 20>, data_count=1, status=CAStatusCode(name='ECA_NORMAL', code=0, code_with_severity=1, severity=<CASeverity.SUCCESS: 1>, success=1, defunct=False, description='Normal successful completion'), subscriptionid=1, metadata=DBR_TIME_DOUBLE(status=<AlarmStatus.NO_ALARM: 0>, severity=<AlarmSeverity.NO_ALARM: 0>, timestamp=1675080768.180643))
[DEBUG][update_thr][06:12:48.183    core.py 542] Update i=109 of ch VARX0AO at 0.0
[DEBUG][  selector][06:12:48.183  client.py 1484] EventAddResponse(data=array([0.9646667]), data_type=<ChannelType.TIME_DOUBLE: 20>, data_count=1, status=CAStatusCode(name='ECA_NORMAL', code=0, code_with_severity=1, severity=<CASeverity.SUCCESS: 1>, success=1, defunct=False, description='Normal successful completion'), subscriptionid=1, metadata=DBR_TIME_DOUBLE(status=<AlarmStatus.NO_ALARM: 0>, severity=<AlarmSeverity.NO_ALARM: 0>, timest

In [ ]:
# # pvinput = EPICSPV(options=PVOptions(name='X0:AI', low=0.0, high=5.0, security=PVAccess.READWRITE))
# # acc.add_pv_object([pvinput])
# pvinput.write(random.random())


In [ ]:
sim.latest_data